In [20]:
!pip install qiskit-aer
# 1. Install the IBM runtime and fake provider expansion
!pip install qiskit-ibm-runtime


In [21]:
# Ensure the runtime is installed
!pip install qiskit-ibm-runtime

from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
from qiskit_ibm_runtime.fake_provider import FakeMarrakesh

# 1. Initialize the 156-qubit Marrakesh backend
# Note: The import name is 'FakeMarrakesh' in the latest runtime
try:
    marrakesh_backend = FakeMarrakesh()
    sim_marrakesh = AerSimulator.from_backend(marrakesh_backend)
    print("✅ Successfully loaded Marrakesh Noise Model!")
except Exception as e:
    print(f"❌ Could not load Marrakesh: {e}")

def run_anonymous_marrakesh(sender_bit):
    qc = QuantumCircuit(4, 4)

    # GHZ State Preparation
    qc.h(0)
    qc.cx(0, 1)
    qc.cx(1, 2)
    qc.cx(2, 3)

    # Alice's Encoding (Z-gate for bit 1)
    if sender_bit == 1:
        qc.z(0)

    # X-basis measurement prep
    qc.h([0, 1, 2, 3])
    qc.measure([0, 1, 2, 3], [0, 1, 2, 3])

    # 2. Transpile for the Heavy-Hex topology
    # Optimization level 3 is essential for mapping to large 156-qubit layouts
    t_qc = transpile(qc, sim_marrakesh, optimization_level=3)

    # 3. Execute with noise
    result = sim_marrakesh.run(t_qc, shots=1).result()
    counts = result.get_counts()

    outcome_str = list(counts.keys())[0]
    bits = [int(b) for b in outcome_str]
    parity = sum(bits) % 2
    return bits, parity

# Test run
bits, parity = run_anonymous_marrakesh(1)
print(f"Marrakesh Result - Bits: {bits}, Parity: {parity}")

✅ Successfully loaded Marrakesh Noise Model!
Marrakesh Result - Bits: [1, 1, 1, 0], Parity: 1
